<a href="https://colab.research.google.com/github/Tessellate-Imaging/Monk_Object_Detection/blob/master/example_notebooks/13_tf_obj_2/Data_Conversion%20-%20COCO%20Type%20to%20VOC%20Type.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# COCO Format

## Dataset Directory Structure

       parent_dir (root_dir)
          |
          |-------ship (coco_dir) 
          |         |
          |         |---Images (img_dir)
          |         |----|
          |              |-------------------img1.jpg
          |              |-------------------img2.jpg
          |              |-------------------.........(and so on)
          |
          |
          |         |---annotations (anno_dir)
          |         |----|
          |              |--------------------instances_Train.json 
          |              |--------------------classes.txt
          
          
 - instances_Train.json -> In proper COCO format
 - classes.txt          -> A list of classes in alphabetical order

# VOC Format

## Dataset Directory Structure

      parent_dir (root_dir)
          |
          |------kangaroo 
          |         |
          |         |---Images (img_dir)
          |         |----|
          |              |-------------------img1.jpg
          |              |-------------------img2.jpg
          |              |-------------------.........(and so on)
          |
          |
          |         |---voc_dir (anno_dir)
          |         |----|
          |              |--------------------img1.xml 
          |              |--------------------img2.xml
          |              |-------------------.........(and so on)
          


In [ ]:
! cp -r Monk_Object_Detection/example_notebooks/sample_dataset/ship .

In [ ]:
! pip install pascal-voc-writer

In [1]:
root_dir = "./";
coco_dir = "ship";
img_dir = "Images";
anno_dir = "annotations";

In [6]:
output_dir = "ship/voc";

In [8]:
import os
if not os.path.isdir(output_dir):
    os.mkdir(output_dir);

In [4]:
anno_file_name = root_dir + "/" + coco_dir + "/" + anno_dir + "/instances_" + img_dir + ".json";

In [5]:
import json
with open(anno_file_name) as json_file:
    data = json.load(json_file)

In [ ]:
from tqdm import tqdm

fnames = [];
hs = [];
ws = [];
labels = [];

for i in tqdm(range(len(data["images"]))):
    fnames.append(data["images"][i]["file_name"]);
    hs.append(data["images"][i]["height"]);
    ws.append(data["images"][i]["width"]);
    labels.append([]);

In [ ]:
classes = [];
for i in tqdm(range(len(data["categories"]))):
    classes.append("");

for i in tqdm(range(len(data["categories"]))):
    classes[data["categories"][i]["id"]] = data["categories"][i]["name"]

In [ ]:
for i in tqdm(range(len(data["annotations"]))):
    bbox = data["annotations"][i]["bbox"];
    category_id = data["annotations"][i]["category_id"];
    img_id = data["annotations"][i]["image_id"];

    x1 = bbox[0];
    y1 = bbox[1];
    x2 = x1 + bbox[2];
    y2 = y1 + bbox[3];
    label = classes[int(category_id)]
    labels[int(img_id)].append([x1, y1, x2, y2, label])

In [ ]:
from pascal_voc_writer import Writer

In [ ]:
for i in tqdm(range(len(labels))):
    img_name = fnames[i];
    img = cv2.imread("ship/images/Train/" + img_name)
    height = hs[i];
    width = ws[i];
    writer = Writer(img_name, width, height)
    for j in range(len(labels[i])):
        x1 = labels[i][j][0]
        y1 = labels[i][j][1]
        x2 = labels[i][j][2]
        y2 = labels[i][j][3]
        label = labels[i][j][4]
        writer.addObject(label, x1, y1, x2, y2);
    
    writer.save(output_dir + "/" + img_name.split(".")[0] + ".xml")